# I. $LU$ factorization of a square matrix

Consider a simple naive implementation of the LU decomposition. 

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [1]:
import numpy as np

def diy_lu(a):
    """Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u

        lam[j+1:, j] = gamma
        L = L @ lam
    return L, u

In [2]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [3]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [4]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  1.110e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -2.819e-16  0.000e+00  0.000e+00  8.080e-04  1.902e-03]
 [ 0.000e+00  3.369e-16  0.000e+00 -1.541e-18  2.168e-19 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16  2.776e-16 -2.776e-16  5.551e-17]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [5]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matix still has full rank, but the naive LU routine breaks down.

In [6]:
np.linalg.matrix_rank(a1)

6

In [7]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

(20% of the grade)

In [8]:
# ... ENTER YOUR CODE HERE ...
def nonzero_test(A):
    p = 0
    for i in range(1,A.shape[0]+1):
        mat = A[:i, :i]
        minor = np.linalg.det(mat)
        if minor == 0:
            p += 1
            break
    if p == 1:
        return False
    else:
        return True

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

(40% of the grade)

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

(40% of the grade)

In [1]:
# ... ENTER YOUR CODE HERE ...
def diy_lup(a):
    N = a.shape[0]
    u = a.copy()
    L = np.eye(N)
    Per_prev = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        arg_remember = np.argmax(abs(u[j,j:]))+j    #   Taking indices of the biggest element
        Per_curr = np.eye(N)
#        max_element = u[j, arg_remember]
        Per_curr[j, j] = 0  #   Constucting the permutation matrix 
        Per_curr[arg_remember,arg_remember] = 0
        Per_curr[j, arg_remember] = 1
        Per_curr[arg_remember, j] = 1
#        print(arg_remember)
#        print(u)
        u = u @ Per_curr.T  # performinf a permutation
#        print(u)
#        print(Per_curr)
#        u[:, arg_remember] = u[:,j]
#        u[:,j] = max_element
#        print(u[j,j], u[j,np.argmax(np.abs(u[j,j:]))],np.max(abs(u[j,j:])) )
        gamma = u[j+1:, j] / u[j, j]
        lam[j+1:, j] = -gamma
        u = lam @ u
        lam[j+1:, j] = gamma
        L = L @ lam
        Per_prev = Per_prev@Per_curr.T
    return L, u, Per_prev

def original_matrix(L, U, P):
    return L @ U @ P.T

In [17]:
L, U, P = diy_lup(a1)
print("test for a: ", nonzero_test(a), "\n","test for a1: ",  nonzero_test(a1), "\n", "original matrix by decomp a1: ", "\n", original_matrix(L, U, P), "\n",  "L = " ,"\n", L, "\n",  "U = ","\n", U, "\n", "Permutation matrix: ","\n",P )

test for a:  True 
 test for a1:  False 
 original matrix by decomp a1:  
 [[3.    3.    3.    3.    3.    3.   ]
 [3.    3.    1.364 1.071 0.882 0.75 ]
 [3.    1.364 0.882 0.652 0.517 0.429]
 [3.    1.071 0.652 0.469 0.366 0.3  ]
 [3.    0.882 0.517 0.366 0.283 0.231]
 [3.    0.75  0.429 0.3   0.231 0.188]] 
 L =  
 [[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.143 1.    0.    0.    0.   ]
 [1.    1.2   1.179 1.    0.    0.   ]
 [1.    1.231 1.294 1.605 1.    0.   ]
 [1.    1.25  1.375 2.007 2.076 1.   ]] 
 U =  
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -2.475e-01 -1.437e-01 -6.259e-02]
 [ 0.000e+00  2.220e-16  2.220e-16 -9.247e-02 -4.755e-02 -1.920e-02]
 [ 0.000e+00 -1.344e-16 -3.565e-16  0.000e+00  1.841e-03  1.181e-03]
 [ 0.000e+00 -1.667e-16  2.941e-16 -1.567e-17 -4.337e-19 -1.233e-05]] 
 Permutation matrix: 